In [65]:
import sqlite3
import numpy as np
import pandas as pd
import keras
import math
import datetime
import tensorflowjs as tfjs
import pickle
import datetime
import calendar
import sklearn.metrics as metrics
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense, LSTM, Dropout, Embedding, Masking
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE

In [43]:
def encodeOutputVariable(y):
    labelencoder_Y_Origin = LabelEncoder()
    y = labelencoder_Y_Origin.fit_transform(y.astype(str))
    return y

def encodeCategoricalData(X, index):
    # encode categorical data
    labelencoder_X_Origin = LabelEncoder()
    X[:, index] = labelencoder_X_Origin.fit_transform(X[:, index].astype(str))
    return X    

def manualEncodeLongStrings(X, column):
    index = 0
    test = 0
    keys = {}
    for row in X:
        key = row[column].replace(", ", "").replace(" ", "")
        if (keys.get(key) == None):
            keys[key] = index
            index += 1
        X[test][column] = keys.get(key)
        test += 1
    return X
    
def defaultMinimumValues(values): 
    for index, x in enumerate(values):
        x = [float(y) if y != None and float(y) > 0.0 else 0 for y in x]
        values[index] = x
    return pd.DataFrame(values)

def encodeHotEncoder(X, categoryIndex):
    # meant to create dummy variables for each category data
    # you only use it for one column at a time, output will be the number of columns
    # needed to represent all discrete values of column
    onehotencoder = OneHotEncoder(categorical_features = [categoryIndex])
    X = defaultMinimumValues(X)
    X = onehotencoder.fit_transform(X.astype(str)).toarray()    
    X = X[:, 1:]
    return X

def determineTotalTime(startDay, startTime, endDay, endTime):
    minsPerDay = 24 * 60
    totalStart = (int(startDay) * minsPerDay) + int(startTime)
    totalEnd = (int(endDay) * minsPerDay) + int(endTime)
    return totalEnd - totalStart

def saveModelThroughPickle(model, filename):
    pickle.dump(model, open(filename, "wb"))

def outputPredictorResults(y_test, y_pred, title):
    # output results for model predictions
    print("For", title, "Classification")
    print(metrics.accuracy_score(y_test, y_pred) * 100)
    print(metrics.confusion_matrix(y_test, y_pred))
    print(pd.crosstab(y_test.ravel(), y_pred.ravel(), rownames=["True"], colnames=["Predicted"], margins=True))
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.zero_one_loss(y_test, y_pred))
#     print(metrics.log_loss(y_test, y_pred))
#    print(metrics.roc_auc_score(y_test, y_pred))    

In [ ]:
class MinimumData(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
    def fit(self, X, y):
        

In [54]:
def createArtificialNeuralNetwork(X_train, y_train, X_test, y_test):
    # create ANN
    print("\nArtificial Neural Network Classifier Section")
    print("---------------------------------")

    # initialize the ann
    classifier = Sequential()

    # adding the input layer and the first hidden layer
    classifier.add(Dense(100, kernel_initializer = "uniform", activation = "relu", input_dim = 41))
    
    classifier.add(Dense(90, kernel_initializer = "uniform", activation="relu"))
    
    classifier.add(Dense(75, kernel_initializer = "uniform", activation="relu"))
    
    # adding the second hidden layer
    classifier.add(Dense(50, kernel_initializer = "uniform", activation = "relu"))

    # adding the third hidden layer
    classifier.add(Dense(25, kernel_initializer = "uniform", activation = "relu"))

    classifier.add(Dense(20, kernel_initializer = "uniform", activation="relu"))
    
    # adding the fourth hidden layer
    classifier.add(Dense(10, kernel_initializer = "uniform", activation = "relu"))
    
    classifier.add(Dense(5, kernel_initializer = "uniform", activation="relu"))

    # adding the output layer 
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))

    # compiling the ANN
    classifier.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])

    # fitting the ANN to the training set
    classifier.fit(X_train, y_train, batch_size = 100, epochs = 200) 
    
    y_pred = classifier.predict(X_test)
    y_pred = (y_pred > 0.5)
    
    # output results
    outputPredictorResults(y_test, y_pred, "Artificial Neural Network")
    
    classifier.save("ann_cause_model.h5")
    classifier.save("ann_cause_model_weights.h5")
    
def creatingRandomForestPredictor(X_train, y_train, X_test, y_test):
    print("\nRandom Forest Classifier Section")
    print("---------------------------------")
    
    # initialize the Random Forest Classifier
    random_forest_classifier = RandomForestClassifier(**{'n_jobs': -1,
        'n_estimators': 1500,
        'warm_start': True, 
        'max_features': 0.5,
        'max_depth': 15,
        'min_samples_leaf': 2,
        'max_features' : 'sqrt',
        'random_state' : 0,
        'verbose': 0
                                                      })
    # run through oversampler simplifier
    oversampler = SMOTE(random_state=0)
    smote_X_train, smote_y_train = oversampler.fit_sample(X_train, y_train)
    
    # fitting Random Forest to the training set
    random_forest_classifier.fit(smote_X_train, smote_y_train)
    
    # Predicting the Test set results
    rf_y_pred = random_forest_classifier.predict(X_test)
    
    # use the threshold of error to determine whether a prediction is valid
    rf_y_pred = (rf_y_pred > 0.5)
    
    # making the confusion matrix
    cm = metrics.confusion_matrix(y_test.ravel(), rf_y_pred.ravel())
    
    print("Training set Score: ", random_forest_classifier.score(X_train, y_train))
    print("Testing set Score: ", random_forest_classifier.score(X_test, y_test)) 
    
    outputPredictorResults(y_test, rf_y_pred, "Random Forest")
    saveModelThroughPickle(random_forest_classifier, "random_forest_cause_model.h5")
    
def createNaiveBayesModel(X_train, y_train, X_test, y_test, preprocess):
    # initialize the Naive Bayes Classifier
    classifier = GaussianNB()
    
    # hook up the preprocess step with the classifier params and create the pipeline
    model = make_pipeline(preprocess, classifier)
    
    # fit the Naive Bayes Classifier to the training set
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    y_pred = (y_pred > 0.5)
    
    print("Training set Score: ", model.score(X_train, y_train))
    print("Testing set Score: ", model.score(X_test, y_test))
    
    outputPredictorResults(y_test, y_pred, "Naive Bayes")
    saveModelThroughPickle(classifier, "naive_bayes_cause_model.h5")

In [55]:
sqlite_file = "./wildfires.sqlite"

# connecting to the database file and saving the select
conn = sqlite3.connect(sqlite_file)
dataset = pd.read_sql_query("select * from Fires limit 5000;", conn)

In [56]:
dataset.iloc[:,:]

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [61]:
# split dataset into train and test lists
filtered_independent = dataset[[
    "STATE", 
    "FIPS_CODE", 
    "LATITUDE", 
    "LONGITUDE", 
    "FIRE_SIZE_CLASS", 
    "FIRE_SIZE", 
    "FIRE_YEAR", 
    "DISCOVERY_DATE",
    "DISCOVERY_TIME",
    "CONT_DATE",
    "CONT_TIME"
]]
filtered_dependent = dataset["STAT_CAUSE_CODE"]
X = filtered_independent.values
y = filtered_dependent.values
filtered_independent.dtypes

STATE               object
FIPS_CODE           object
LATITUDE           float64
LONGITUDE          float64
FIRE_SIZE_CLASS     object
FIRE_SIZE          float64
FIRE_YEAR            int64
DISCOVERY_DATE     float64
DISCOVERY_TIME      object
CONT_DATE          float64
CONT_TIME           object
dtype: object

In [69]:
# using the ColumnTransformer only approach
numerical_features = filtered_independent.dtypes != "object"
categorical_features = ~numerical_features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

preprocess = make_column_transformer (
    (categorical_features, OneHotEncoder()),
    (categorical_features, defaultMinimumValues),
    (numerical_features, make_pipeline(SimpleImputer(), StandardScaler()))
)
process_columns = filtered_independent.columns == "STATE"
process_columns

array([ True, False, False, False, False, False, False, False, False,
       False, False])

In [63]:
createNaiveBayesModel(X_train, y_train, X_test, y_test, preprocess)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/ops.py:792: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [10]:
createArtificialNeuralNetwork(X_train, y_train, X_test, y_test)


Artificial Neural Network Classifier Section
---------------------------------
Epoch 1/200
3500/3500 [==============================] - 1s 172us/step - loss: 13.7945 - acc: 0.5220
Epoch 2/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 3/200
3500/3500 [==============================] - 0s 21us/step - loss: 13.2523 - acc: 0.5220
Epoch 4/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 5/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 6/200
3500/3500 [==============================] - 0s 21us/step - loss: 13.2523 - acc: 0.5220
Epoch 7/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 8/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 9/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 10/200
3500/3

3500/3500 [==============================] - 0s 23us/step - loss: 13.2523 - acc: 0.5220
Epoch 162/200
3500/3500 [==============================] - 0s 22us/step - loss: 13.2523 - acc: 0.5220
Epoch 163/200
3500/3500 [==============================] - 0s 22us/step - loss: 13.2523 - acc: 0.5220
Epoch 164/200
3500/3500 [==============================] - 0s 21us/step - loss: 13.2523 - acc: 0.5220
Epoch 165/200
3500/3500 [==============================] - 0s 19us/step - loss: 13.2523 - acc: 0.5220
Epoch 166/200
3500/3500 [==============================] - 0s 22us/step - loss: 13.2523 - acc: 0.5220
Epoch 167/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 168/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 169/200
3500/3500 [==============================] - 0s 20us/step - loss: 13.2523 - acc: 0.5220
Epoch 170/200
3500/3500 [==============================] - 0s 21us/step - loss: 13.2523 - acc: 0

/Users/vincentdu/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
creatingRandomForestPredictor(X_train, y_train, X_test, y_test)


Random Forest Classifier Section
---------------------------------
Training set Score:  0.8265714285714286
Testing set Score:  0.6293333333333333
For Random Forest Classification
51.0
[[765   0   0   0   0   0   0   0   0]
 [ 53   0   0   0   0   0   0   0   0]
 [ 25   0   0   0   0   0   0   0   0]
 [253   0   0   0   0   0   0   0   0]
 [ 74   0   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0   0]
 [124   0   0   0   0   0   0   0   0]
 [ 15   0   0   0   0   0   0   0   0]
 [185   0   0   0   0   0   0   0   0]]
Predicted  True   All
True                 
1.0         765   765
2.0          53    53
3.0          25    25
4.0         253   253
5.0          74    74
6.0           6     6
7.0         124   124
8.0          15    15
9.0         185   185
All        1500  1500
             precision    recall  f1-score   support

        1.0       0.51      1.00      0.68       765
        2.0       0.00      0.00      0.00        53
        3.0       0.00      0.00      0

/Users/vincentdu/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
